# Explore here

In [1]:
# Your code here
import os
from sqlalchemy import create_engine, text
import pandas as pd
from dotenv import load_dotenv

In [5]:
load_dotenv()

def connect():
    global engine # Esto nos permite usar una variable global llamada "engine"
    # Un "connection string" es básicamente una cadena que contiene todas las credenciales de la base de datos juntas
    connection_string = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
    print("Starting the connection...")
    engine = create_engine(connection_string)
    engine.connect()
    return engine

engine = connect()

try:
    with engine.connect() as conn:
        # Ejecuta cada sentencia CREATE TABLE individualmente
        statements = [
            """
            CREATE TABLE publishers(
                publisher_id INT NOT NULL,
                name VARCHAR(255) NOT NULL,
                PRIMARY KEY(publisher_id)
            );
            """,
            """
            CREATE TABLE authors(
                author_id INT NOT NULL,
                first_name VARCHAR(100) NOT NULL,
                middle_name VARCHAR(50) NULL,
                last_name VARCHAR(100) NULL,
                PRIMARY KEY(author_id)
            );
            """,
            """
            CREATE TABLE books(
                book_id INT NOT NULL,
                title VARCHAR(255) NOT NULL,
                total_pages INT NULL,
                rating DECIMAL(4, 2) NULL,
                isbn VARCHAR(13) NULL,
                published_date DATE,
                publisher_id INT NULL,
                PRIMARY KEY(book_id),
                CONSTRAINT fk_publisher FOREIGN KEY(publisher_id) REFERENCES publishers(publisher_id)
            );
            """,
            """
            CREATE TABLE book_authors (
                book_id INT NOT NULL,
                author_id INT NOT NULL,
                PRIMARY KEY(book_id, author_id),
                CONSTRAINT fk_book FOREIGN KEY(book_id) REFERENCES books(book_id) ON DELETE CASCADE,
                CONSTRAINT fk_author FOREIGN KEY(author_id) REFERENCES authors(author_id) ON DELETE CASCADE
            );
            """
        ]
        for statement in statements:
            conn.execute(text(statement))  # Encapsula cada sentencia con text()

        conn.commit()  # Confirma los cambios después de crear todas las tablas
        print("¡Tablas creadas exitosamente!")

        data = {
            "publishers": [
                {"publisher_id": 1, "name": 'O Reilly Media'},
                {"publisher_id": 2, "name": 'A Book Apart'},
                {"publisher_id": 3, "name": 'A K PETERS'},
                {"publisher_id": 4, "name": 'Academic Press'},
                {"publisher_id": 5, "name": 'Addison Wesley'},
                {"publisher_id": 6, "name": 'Albert&Sweigart'},
                {"publisher_id": 7, "name": 'Alfred A. Knopf'}
            ],
            "authors": [
                {"author_id": 1, "first_name": 'Merritt', "middle_name": None, "last_name": 'Eric'},
                {"author_id": 2, "first_name": 'Linda', "middle_name": None, "last_name": 'Mui'},
                # ... (resto de los datos de autores)
                {"author_id": 8, "first_name": 'Paul', "middle_name": None, "last_name": 'Albitz'}
            ],
            "books": [
                {"book_id": 1, "title": 'Lean Software Development: An Agile Toolkit', "total_pages": 240, "rating": 4.17, "isbn": '9780320000000', "published_date": '2003-05-18', "publisher_id": 5},
                {"book_id": 2, "title": 'Facing the Intelligence Explosion', "total_pages": 91, "rating": 3.87, "isbn": None, "published_date": '2013-02-01', "publisher_id": 7},
                # ... (resto de los datos de libros)
                {"book_id": 10, "title": 'Minds and Computers: An Introduction to the Philosophy of Artificial Intelligence', "total_pages": 222, "rating": 3.54, "isbn": '9780750000000', "published_date": '2007-02-13', "publisher_id": 7},
            ],
            "book_authors": [
                {"book_id": 1, "author_id": 1},
                {"book_id": 2, "author_id": 8},
                # ... (resto de los datos de book_authors)
                {"book_id": 10, "author_id": 1}
            ]
        }
        for table_name, rows in data.items():  # Itera sobre las tablas y sus datos
            if rows:  # Verifica si hay datos para la tabla
                columns = list(rows[0].keys())  # Obtiene los nombres de las columnas
                placeholders = ", ".join(["?"] * len(columns))
                query = text(f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES ({placeholders})")

                for row in rows:
                    values = [row[col] for col in columns]  # Extrae los valores en el orden correcto
                    conn.execute(query, *values)  # Ejecuta la query

        conn.commit()
        print("Datos insertados exitosamente!")

except Exception as e:
    print(f"Ocurrió un error: {e}")
    conn.rollback()

finally:
    engine.dispose()

Starting the connection...


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  role "'" does not exist

(Background on this error at: https://sqlalche.me/e/20/e3q8)